# LangChain: Agents

## Outline:

* Using built in LangChain tools: DuckDuckGo search and Wikipedia
* Defining your own tools

In [5]:
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

import warnings
warnings.filterwarnings("ignore")

## Built-in LangChain tools

In [3]:
#!pip install -U wikipedia
#!pip install langchain_experimental

In [8]:
# from langchain.agents import create_python_agent
from langchain.agents import load_tools, initialize_agent
from langchain.agents import AgentType
# from langchain.tools.python.tool import PythonREPLTool
# from langchain.python import PythonREPL
from langchain_openai import AzureChatOpenAI
from langchain.agents import AgentExecutor
from langchain_experimental.tools import PythonREPLTool
from langchain.agents import create_openai_functions_agent
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

In [9]:
llm = AzureChatOpenAI(temperature=0.0, openai_api_version="2023-12-01-preview", azure_deployment="gpt-35-turbo-felix", openai_api_key=os.getenv("AZURE_OPENAI_KEY"))

In [8]:
tools = load_tools(["llm-math","wikipedia"], llm=llm)

In [9]:
agent= initialize_agent(
    tools, 
    llm, 
    agent=AgentType.CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    handle_parsing_errors=True,
    verbose = True)

In [10]:
agent("What is the 25% of 300?")



> Entering new AgentExecutor chain...
Thought: I need to use a calculator to find the answer to this math problem.

Action:
```
{
  "action": "Calculator",
  "action_input": "300*0.25"
}
```


Observation: Answer: 75.0
Thought:I have successfully calculated the answer to the math problem.

Final Answer: 75.0

> Finished chain.


{'input': 'What is the 25% of 300?', 'output': '75.0'}

In [11]:
question = "Tom M. Mitchell is an American computer scientist \
and the Founders University Professor at Carnegie Mellon University (CMU)\
what book did he write?"
result = agent(question) 



> Entering new AgentExecutor chain...
Thought: I should use the wikipedia tool to search for Tom M. Mitchell's bibliography.

Action:
```
{
  "action": "wikipedia",
  "action_input": "Tom M. Mitchell"
}
```



ConnectTimeout: HTTPConnectionPool(host='en.wikipedia.org', port=80): Max retries exceeded with url: /w/api.php?list=search&srprop=&srlimit=3&limit=3&srsearch=Tom+M.+Mitchell&format=json&action=query (Caused by ConnectTimeoutError(<urllib3.connection.HTTPConnection object at 0x127ea1a90>, 'Connection to en.wikipedia.org timed out. (connect timeout=None)'))

## Python Agent

In [44]:
# prompt = ChatPromptTemplate.from_messages(
#     [
#         (
#             "system",
#             "You are very powerful assistant, but don't know current events, You have access to a python REPL, which you can use to execute python code.",
#         ),
#         ("user", "{input}"),
#         MessagesPlaceholder(variable_name="agent_scratchpad"),
#     ]
# )

instructions = """You are an agent designed to write and execute python code to answer questions.
You have access to a python REPL, which you can use to execute python code.
If you get an error, debug your code and try again.
Only use the output of your code to answer the question. 
You might know the answer without running any code, but you should still run the code to get the answer.
If it does not seem like you can write code to answer the question, just return "I don't know" as the answer.
"""
base_prompt = hub.pull("langchain-ai/openai-functions-template")
prompt = base_prompt.partial(instructions=instructions)

In [13]:

from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.agents.format_scratchpad.openai_tools import (
    format_to_openai_tool_messages,
)
from langchain.agents.output_parsers.openai_tools import OpenAIToolsAgentOutputParser
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are very powerful assistant, but bad at calculating lengths of words.",
        ),
        ("user", "{input}"),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ]
)

tools = [PythonREPLTool()]
llm_with_tools = llm.bind_tools(tools)

agent = (
    {
        "input": lambda x: x["input"],
        "agent_scratchpad": lambda x: format_to_openai_tool_messages(
            x["intermediate_steps"]
        ),
    }
    | prompt
    | llm_with_tools
    | OpenAIToolsAgentOutputParser()
)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)
prinnt(agent_executor.execute())


In [14]:
# list(agent_executor.stream({"input": "How many letters in the word eudca"}))
agent_executor.invoke({"input": "What is the 10th fibonacci number?"})



> Entering new AgentExecutor chain...


ValueError: An output parsing error occurred. In order to pass this error back to the agent and have it try again, pass `handle_parsing_errors=True` to the AgentExecutor. This is the error: Could not parse tool input: {'arguments': '// Using the formula for the nth fibonacci number: F(n) = (phi^n - (1-phi)^n) / sqrt(5), where phi is the golden ratio (1 + sqrt(5)) / 2\nconst n = 10;\nconst phi = (1 + Math.sqrt(5)) / 2;\nconst fib = Math.round((phi ** n - (1 - phi) ** n) / Math.sqrt(5));\nfib;', 'name': 'python'} because the `arguments` is not valid JSON.

In [45]:
tools = [PythonREPLTool()]
agent = create_openai_functions_agent(
    llm,
    tools,
    prompt
)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [35]:
customer_list = [["Harrison", "Chase"], 
                 ["Lang", "Chain"],
                 ["Dolly", "Too"],
                 ["Elle", "Elem"], 
                 ["Geoff","Fusion"], 
                 ["Trance","Former"],
                 ["Jen","Ayai"]
                ]

In [46]:
# langchain.debug=True
agent_executor.invoke({"input": "What is the 10th fibonacci number?"})
# agent_executor.invoke({"input": f"""Sort these customers by \
# last name and then first name \
# and print the output: {customer_list}"""}) 



> Entering new AgentExecutor chain...


ValueError: An output parsing error occurred. In order to pass this error back to the agent and have it try again, pass `handle_parsing_errors=True` to the AgentExecutor. This is the error: Could not parse tool input: {'arguments': "# We can calculate the 10th fibonacci number by using a loop or recursion.\n# Here's a loop implementation:\n\ndef fibonacci(n):\n    if n <= 1:\n        return n\n    else:\n        a, b = 0, 1\n        for i in range(n-1):\n            a, b = b, a + b\n        return b\n\nfibonacci(10)", 'name': 'python'} because the `arguments` is not valid JSON.

#### View detailed outputs of the chains

In [ ]:
import langchain
langchain.debug=True
agent.run(f"""Sort these customers by \
last name and then first name \
and print the output: {customer_list}""") 
langchain.debug=False

## Define your own tool

In [ ]:
#!pip install DateTime

In [ ]:
from langchain.agents import tool
from datetime import date

In [ ]:
@tool
def time(text: str) -> str:
    """Returns todays date, use this for any \
    questions related to knowing todays date. \
    The input should always be an empty string, \
    and this function will always return todays \
    date - any date mathmatics should occur \
    outside this function."""
    return str(date.today())

In [ ]:
agent= initialize_agent(
    tools + [time], 
    llm, 
    agent=AgentType.CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    handle_parsing_errors=True,
    verbose = True)

**Note**: 

The agent will sometimes come to the wrong conclusion (agents are a work in progress!). 

If it does, please try running it again.

In [ ]:
try:
    result = agent("whats the date today?") 
except: 
    print("exception on external access")